In [1]:
from transformers import pipeline

# Initialize the pipeline and tokenizer once
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

def extract_text_triplets(input_text):
    """
    Extracts triplets from the given text.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail'.
    """
    # Use the tokenizer manually since we need special tokens
    extracted_text = triplet_extractor.tokenizer.batch_decode([
        triplet_extractor(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]
    ])

    # Function to parse the generated text and extract the triplets
    def extract_triplets(text):
        triplets = []
        relation, subject, object_ = '', '', ''
        text = text.strip()
        current = 'x'
        for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
            if token == "<triplet>":
                current = 't'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                    relation = ''
                subject = ''
            elif token == "<subj>":
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                object_ = ''
            elif token == "<obj>":
                current = 'o'
                relation = ''
            else:
                if current == 't':
                    subject += ' ' + token
                elif current == 's':
                    object_ += ' ' + token
                elif current == 'o':
                    relation += ' ' + token
        if subject != '' and relation != '' and object_ != '':
            triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
        return triplets

    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets

# Example usage


/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
input_text = "Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic"
extracted_triplets = extract_text_triplets(input_text)


In [9]:
print(extracted_triplets)

[{'head': 'Punta Cana', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Higuey', 'type': 'located in the administrative territorial entity', 'tail': 'La Altagracia Province'}, {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'La Altagracia Province', 'type': 'country', 'tail': 'Dominican Republic'}, {'head': 'Dominican Republic', 'type': 'contains administrative territorial entity', 'tail': 'La Altagracia Province'}]


In [12]:
t1 = "obama is president of usa."
extracted_triplets1 = extract_text_triplets(t1)

In [13]:
extracted_triplets1

[{'head': 'Obama', 'type': 'position held', 'tail': 'president of usa'},
 {'head': 'president of usa', 'type': 'officeholder', 'tail': 'Obama'}]

In [2]:
from llama_cpp import Llama
import re
import json
from IPython.core.display import display, HTML
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

/var/folders/_b/ccc66ybs0m59z0yyhly4_jp80000gn/T/ipykernel_10502/661250872.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
llm = Llama(model_path="/Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf",  
n_ctx=2048,
n_gpu_layers=-1,
n_batch=512,
callback_manager=callback_manager,
verbose=True,)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dime

In [18]:
question = "What is first day of week?"

In [19]:
output = llm(question, max_tokens=100)


llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      30.11 ms /   100 runs   (    0.30 ms per token,  3321.27 tokens per second)
llama_print_timings: prompt eval time =     524.39 ms /     8 tokens (   65.55 ms per token,    15.26 tokens per second)
llama_print_timings:        eval time =    7763.41 ms /    99 runs   (   78.42 ms per token,    12.75 tokens per second)
llama_print_timings:       total time =    8777.73 ms /   107 tokens


In [20]:
output

{'id': 'cmpl-8a4d6f8f-3e77-4555-ba87-c27aa2204de9',
 'object': 'text_completion',
 'created': 1709487985,
 'model': '/Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf',
 'choices': [{'text': "\n\nThe first day of the week is traditionally considered to be Sunday, according to both the Christian and the Jewish calendars. This is based on the biblical account of creation in the Book of Genesis, where God rested on the seventh day (Saturday) after having created the world in six days. As a result, Sunday became known as the 'Lord's Day', a day of rest and worship. However, it's important to note that different cultures and",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 8, 'completion_tokens': 100, 'total_tokens': 108}}

In [21]:
from transformers import pipeline

# Initialize the pipeline and tokenizer once
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

def extract_text_triplets(input_text):
    """
    Extracts triplets from the given text.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail'.
    """
    # Use the tokenizer manually since we need special tokens
    extracted_text = triplet_extractor.tokenizer.batch_decode([
        triplet_extractor(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]
    ])

    # Function to parse the generated text and extract the triplets
    def extract_triplets(text):
        triplets = []
        relation, subject, object_ = '', '', ''
        text = text.strip()
        current = 'x'
        for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
            if token == "<triplet>":
                current = 't'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                    relation = ''
                subject = ''
            elif token == "<subj>":
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                object_ = ''
            elif token == "<obj>":
                current = 'o'
                relation = ''
            else:
                if current == 't':
                    subject += ' ' + token
                elif current == 's':
                    object_ += ' ' + token
                elif current == 'o':
                    relation += ' ' + token
        if subject != '' and relation != '' and object_ != '':
            triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
        return triplets

    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets

In [22]:
input_text = "Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic"
extracted_triplets = extract_text_triplets(input_text)

In [23]:
extracted_triplets

[{'head': 'Punta Cana',
  'type': 'located in the administrative territorial entity',
  'tail': 'La Altagracia Province'},
 {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'},
 {'head': 'Higuey',
  'type': 'located in the administrative territorial entity',
  'tail': 'La Altagracia Province'},
 {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'},
 {'head': 'La Altagracia Province',
  'type': 'country',
  'tail': 'Dominican Republic'},
 {'head': 'Dominican Republic',
  'type': 'contains administrative territorial entity',
  'tail': 'La Altagracia Province'}]

In [11]:
prompt_template = '''<s>[INST] <<SYS>>
Assistant is an expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

The available action to Assistant is:

- "extract_text_triplets": Useful for when Assistant is asked to extract triplets from a given text.
  - To use the extract_triplets tool, Assistant should respond like so:
    {{"action": "extract_text_triplets", "action_input": "Your text here"}}


Here are the entities and relations and their corresponding mappings.
"entities": {"Loc": {"short": "Loc", "verbose": "Location"}, "Org": {"short": "Org", "verbose": "Organization"}, "Peop": {"short": "Peop", "verbose":"People"}, "Other": {"short": "Other", "verbose": "Other"}}, 
"relations": {"Work_For": {"short": "Work", "verbose": "Work for", "symmetric": false}, "Kill": {"short": "Kill", "verbose": "Kill", "symmetric": false}, "OrgBased_In": {"short": "OrgBI", "verbose": "Organization based in", "symmetric": false}, "Live_In": {"short": "Live", "verbose": "Live in", "symmetric": false}, "Located_In": {"short": "LocIn", "verbose": "Located in", "symmetric": false}}}
mapping = {'Kill': 'killed by', 'Live_In': 'residence', 'Located_In': 'location', 'OrgBased_In': 'headquarters location', 'Work_For': 'employer'}
mapping_types = {'Peop': '<peop>', 'Org': '<org>', 'Other': '<other>', 'Loc': '<loc>'}

When extracting triples make sure that head and tail entities are from mapping dictionary and relations are from mapping_types dictionary only


Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: I'm good thanks, how are you?
User: Can you extract all the triplets from this text: "Gràcia is a district of the city of Barcelona, Spain."
Assistant: {{"action": "extract_text_triplets", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}}
User: Also give triples for "obama was US president"
Assistant: {{"action": "extract_text_triplets", "action_input": "obama was US president"}}


<</SYS>>

{0}[/INST]'''

In [5]:
def process_command1(command):
    # Put user command into prompt
    prompt = prompt_template.format("User: " + command)
    # Send command to the model
    output = llm(prompt, max_tokens=2000, stop=["User:"])
    response = output['choices'][0]['text']
    #print(type(response))
    firstBracketIndex = response.index("{")
    lastBracketIndex = len(response) - response[::-1].index("}")
    print(firstBracketIndex)
    return response

In [50]:
process_command1("Can you please give triple for \"Ananya is a good girl\"")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =       7.28 ms /    26 runs   (    0.28 ms per token,  3573.39 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2541.63 ms /    26 runs   (   97.75 ms per token,    10.23 tokens per second)
llama_print_timings:       total time =    2691.16 ms /    27 tokens


1


' {"action": "extract_text_triplets", "action_input": "Ananya is a good girl"}'

In [6]:
def process_command(command):
    # Put user command into prompt
    prompt = prompt_template.format("User: " + command)
    # Send command to the model
    output = llm(prompt, max_tokens=2000, stop=["User:"])
    response = output['choices'][0]['text']

    # try to find json in the response
    try:
        # Extract json from model response by finding first and last brackets {}
        firstBracketIndex = response.index("{")
        lastBracketIndex = len(response) - response[::-1].index("}")
        jsonString = response[firstBracketIndex:lastBracketIndex]
        responseJson = json.loads(jsonString)
        if responseJson['action'] == 'extract_text_triplets':
            extracted_triplets = extract_text_triplets(responseJson['action_input'])
            return extracted_triplets   
    except Exception as e:
        print(e)
    # No json match, just return response
    return response


In [56]:
process_command("Can you please give triple for \"Ananya is a good girl\"")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =       6.08 ms /    26 runs   (    0.23 ms per token,  4277.02 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2339.56 ms /    26 runs   (   89.98 ms per token,    11.11 tokens per second)
llama_print_timings:       total time =    2447.47 ms /    27 tokens


[{'head': 'Ananya', 'type': 'instance of', 'tail': 'good girl'}]

In [54]:
extracted_triplets = process_command("Can you please give triple for \"Ananya is a good girl\"")

# Save the extracted triples to a JSON file
output_file_path = 'out.json'  # Define the output file path

# Write the extracted triples to the output file
with open(output_file_path, 'w') as file:
    json.dump(extracted_triplets, file, indent=4)

# Print a message to indicate that the file has been saved
print(f"Extracted triples have been saved to {output_file_path}")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =       5.24 ms /    26 runs   (    0.20 ms per token,  4961.83 tokens per second)
llama_print_timings: prompt eval time =     847.20 ms /    16 tokens (   52.95 ms per token,    18.89 tokens per second)
llama_print_timings:        eval time =    2072.23 ms /    25 runs   (   82.89 ms per token,    12.06 tokens per second)
llama_print_timings:       total time =    3088.59 ms /    41 tokens


hello_op
Extracted triples have been saved to out.json


In [7]:
process_command("Can you give triple for: Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic")


llama_print_timings:        load time =    3500.53 ms
llama_print_timings:      sample time =      15.87 ms /    56 runs   (    0.28 ms per token,  3528.45 tokens per second)
llama_print_timings: prompt eval time =    3499.73 ms /   335 tokens (   10.45 ms per token,    95.72 tokens per second)
llama_print_timings:        eval time =    4712.87 ms /    55 runs   (   85.69 ms per token,    11.67 tokens per second)
llama_print_timings:       total time =    8463.30 ms /   390 tokens


[{'head': 'Punta Cana',
  'type': 'located in the administrative territorial entity',
  'tail': 'La Altagracia Province'},
 {'head': 'Punta Cana', 'type': 'country', 'tail': 'Dominican Republic'},
 {'head': 'Higuey',
  'type': 'located in the administrative territorial entity',
  'tail': 'La Altagracia Province'},
 {'head': 'Higuey', 'type': 'country', 'tail': 'Dominican Republic'},
 {'head': 'La Altagracia Province',
  'type': 'country',
  'tail': 'Dominican Republic'},
 {'head': 'Dominican Republic',
  'type': 'contains administrative territorial entity',
  'tail': 'La Altagracia Province'}]

In [8]:
context = "Twenty-five years after the assassination of President John F. Kennedy , Lee Harvey Oswald 's widow says she now believes Oswald did not act alone in the killing ."

In [9]:
command = f"Can you please give triple for \"{context}\""

In [10]:
process_command(command)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3500.53 ms
llama_print_timings:      sample time =      19.74 ms /    60 runs   (    0.33 ms per token,  3039.67 tokens per second)
llama_print_timings: prompt eval time =     829.69 ms /    49 tokens (   16.93 ms per token,    59.06 tokens per second)
llama_print_timings:        eval time =    4673.44 ms /    59 runs   (   79.21 ms per token,    12.62 tokens per second)
llama_print_timings:       total time =    5842.80 ms /   108 tokens


[{'head': 'assassination of President John F. Kennedy',
  'type': 'participant',
  'tail': 'Lee Harvey Oswald'},
 {'head': 'Lee Harvey Oswald',
  'type': 'significant event',
  'tail': 'assassination of President John F. Kennedy'}]

In [58]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you please give triple for \"{context}\""
        # Use the process_command function to predict the extracted triples
        extracted_triplets = process_command(command)
        # Append the extracted triples to the item under the 'triples' key
        item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
        modified_data.append(item)
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.dump(modified_data, file, indent=4)

# Example usage
eval_file_path = '/Users/ananyahooda/Desktop/Text2kg/shot_col.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/Text2kg/out.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      20.31 ms /    60 runs   (    0.34 ms per token,  2953.77 tokens per second)
llama_print_timings: prompt eval time =     817.35 ms /    45 tokens (   18.16 ms per token,    55.06 tokens per second)
llama_print_timings:        eval time =    4720.26 ms /    59 runs   (   80.00 ms per token,    12.50 tokens per second)
llama_print_timings:       total time =    5852.14 ms /   104 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      19.52 ms /    60 runs   (    0.33 ms per token,  3074.56 tokens per second)
llama_print_timings: prompt eval time =     666.03 ms /    44 tokens (   15.14 ms per token,    66.06 tokens per second)
llama_print_timings:        eval time =    4699.43 ms /    59 runs   (   79.65 ms per token,    12.55 tokens per second)
llama_print_timings:       to

In [59]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you please give triple for \"{context}\""
        # Use the process_command function to predict the extracted triples
        extracted_triplets = process_command(command)
        # Append the extracted triples to the item under the 'triples' key
        item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
        modified_data.append(item)
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.dump(modified_data, file, indent=4)

# Example usage
eval_file_path = '/Users/ananyahooda/Desktop/Text2kg/conll04_eval.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/Text2kg/pred_conll04.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      15.49 ms /    51 runs   (    0.30 ms per token,  3291.60 tokens per second)
llama_print_timings: prompt eval time =     810.37 ms /    35 tokens (   23.15 ms per token,    43.19 tokens per second)
llama_print_timings:        eval time =    3986.93 ms /    50 runs   (   79.74 ms per token,    12.54 tokens per second)
llama_print_timings:       total time =    5033.26 ms /    85 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      81.57 ms /   217 runs   (    0.38 ms per token,  2660.16 tokens per second)
llama_print_timings: prompt eval time =     387.39 ms /    13 tokens (   29.80 ms per token,    33.56 tokens per second)
llama_print_timings:        eval time =   17097.64 ms /   216 runs   (   79.16 ms per token,    12.63 tokens per second)
llama_print_timings:       to

Extra data: line 3 column 1 (char 92)



llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      62.42 ms /   163 runs   (    0.38 ms per token,  2611.43 tokens per second)
llama_print_timings: prompt eval time =     318.34 ms /    30 tokens (   10.61 ms per token,    94.24 tokens per second)
llama_print_timings:        eval time =   13059.39 ms /   162 runs   (   80.61 ms per token,    12.40 tokens per second)
llama_print_timings:       total time =   14293.15 ms /   192 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      19.82 ms /    31 runs   (    0.64 ms per token,  1564.16 tokens per second)
llama_print_timings: prompt eval time =     401.60 ms /    15 tokens (   26.77 ms per token,    37.35 tokens per second)
llama_print_timings:        eval time =    2395.61 ms /    30 runs   (   79.85 ms per token,    12.52 tokens per second)
llama_print_timings:       total time =    3033.20 ms /    45 

Invalid \escape: line 1 column 44 (char 43)



llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =     240.03 ms /   301 runs   (    0.80 ms per token,  1254.04 tokens per second)
llama_print_timings: prompt eval time =     728.66 ms /    54 tokens (   13.49 ms per token,    74.11 tokens per second)
llama_print_timings:        eval time =   24584.80 ms /   300 runs   (   81.95 ms per token,    12.20 tokens per second)
llama_print_timings:       total time =   28234.06 ms /   354 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      52.08 ms /    63 runs   (    0.83 ms per token,  1209.75 tokens per second)
llama_print_timings: prompt eval time =     671.60 ms /    47 tokens (   14.29 ms per token,    69.98 tokens per second)
llama_print_timings:        eval time =    4877.33 ms /    62 runs   (   78.67 ms per token,    12.71 tokens per second)
llama_print_timings:       total time =    6120.80 ms /   109 

...


llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      55.28 ms /   148 runs   (    0.37 ms per token,  2677.28 tokens per second)
llama_print_timings: prompt eval time =     592.15 ms /    60 tokens (    9.87 ms per token,   101.33 tokens per second)
llama_print_timings:        eval time =   11534.60 ms /   147 runs   (   78.47 ms per token,    12.74 tokens per second)
llama_print_timings:       total time =   13024.22 ms /   207 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      18.18 ms /    49 runs   (    0.37 ms per token,  2695.57 tokens per second)
llama_print_timings: prompt eval time =     661.90 ms /    37 tokens (   17.89 ms per token,    55.90 tokens per second)
llama_print_timings:        eval time =    3721.64 ms /    48 runs   (   77.53 ms per token,    12.90 tokens per second)
llama_print_timings:       total time =    4670.16 ms /    85 

Extra data: line 3 column 1 (char 153)



llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      16.31 ms /    46 runs   (    0.35 ms per token,  2820.36 tokens per second)
llama_print_timings: prompt eval time =     318.10 ms /    31 tokens (   10.26 ms per token,    97.45 tokens per second)
llama_print_timings:        eval time =    3530.55 ms /    45 runs   (   78.46 ms per token,    12.75 tokens per second)
llama_print_timings:       total time =    4116.20 ms /    76 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      51.98 ms /   214 runs   (    0.24 ms per token,  4116.81 tokens per second)
llama_print_timings: prompt eval time =     654.19 ms /    36 tokens (   18.17 ms per token,    55.03 tokens per second)
llama_print_timings:        eval time =   18889.28 ms /   213 runs   (   88.68 ms per token,    11.28 tokens per second)
llama_print_timings:       total time =   20430.54 ms /   249 

substring not found



llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =       9.63 ms /    35 runs   (    0.28 ms per token,  3633.72 tokens per second)
llama_print_timings: prompt eval time =     311.44 ms /    20 tokens (   15.57 ms per token,    64.22 tokens per second)
llama_print_timings:        eval time =    2871.13 ms /    34 runs   (   84.45 ms per token,    11.84 tokens per second)
llama_print_timings:       total time =    3349.46 ms /    54 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     524.70 ms
llama_print_timings:      sample time =      20.32 ms /    58 runs   (    0.35 ms per token,  2855.03 tokens per second)
llama_print_timings: prompt eval time =     659.38 ms /    43 tokens (   15.33 ms per token,    65.21 tokens per second)
llama_print_timings:        eval time =    4451.87 ms /    57 runs   (   78.10 ms per token,    12.80 tokens per second)
llama_print_timings:       total time =    5432.69 ms /   100 